In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os

In [2]:
def extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff=5000):
    I_left = np.array([])
    I_right = np.array([])
    bad_indices = np.array([])
    times = np.linspace(0, 90, 270)

    for filename in sorted(os.listdir(reduced_folder), key = extract_number):
        if filename.startswith(reduced_filename):
            with fits.open(os.path.join(reduced_folder, filename)) as hdul:
                reduced_img_data = hdul[0].data
                ys, xs, = np.indices(reduced_img_data.shape)
                lradius = np.sqrt((ys-lcenter[0])**2+(xs-lcenter[1])**2)
                rradius = np.sqrt((ys-rcenter[0])**2+(xs-rcenter[1])**2)

                lbackground_mask = (lradius > 20) & (lradius < 26)
                rbackground_mask = (rradius > 20) & (rradius < 26)   # Index the background around each spot, take the median value

                background_lmedian = np.median(reduced_img_data[lbackground_mask])
                background_rmedian = np.median(reduced_img_data[rbackground_mask])

                lflux = np.sum(reduced_img_data[lradius < maxradius] - background_lmedian)   # Now take the flux with the background mask subtracted
                rflux = np.sum(reduced_img_data[rradius < maxradius] - background_rmedian)
                I_left = np.append(I_left, lflux)
                I_right = np.append(I_right, rflux)

                if lflux+rflux < cutoff:
                    print("Warning: low flux detected, check the image " + filename + ", index: " + str(sorted(os.listdir(reduced_folder), key = extract_number).index(filename)))
                    bad_indices = np.append(bad_indices, sorted(os.listdir(reduced_folder), key = extract_number).index(filename))
                else:
                    continue 

    # Makes the array a list of integers that can be used to index the other array
    bad_indices = bad_indices.astype(int)
    # Deletes the bad indices (caused by camera glitch or some other complication) from the data
    I_left = np.delete(I_left, bad_indices)
    I_right = np.delete(I_right, bad_indices)
    new_times = np.delete(times, bad_indices)

    return I_left, I_right, new_times, bad_indices

In [ ]:
reduced_filename = 'Reduced_SuperK_'
reduced_folder = "/home/shared/exoserver/Lab_Data/Mueller_Matrix_Polarimeter/SuperK_Intensity/Reduced_SuperK_Intensity/First_Attempt_Reduced/"
lcenter = [316, 247]
rcenter = [316, 324]
maxradius = 20
cutoff = 500

extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff)
Il = extracted_data[0]
Ir = extracted_data[1]
times = extracted_data[2]

I = (Il+Ir)

In [ ]:
plt.ylabel('Intensity (counts)', fontsize=12, fontweight='bold')
plt.xlabel('Time (minutes)', fontsize=12, fontweight='bold')
plt.plot(times, I, marker='o', linestyle='-', label="SuperK Laser Intensity")
plt.legend()

folder_path = "/home/shared/exoserver/Lab_Data/Mueller_Matrix_Polarimeter/SuperK_Intensity"
file_name = 'SuperK_Laser_Stability.png'
#plt.savefig(folder_path + file_name, bbox_inches='tight', dpi=300)